In [ ]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method



"""


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2021-01-03 23:16:12,970 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-03 23:16:12,971 - settings - Setting database.user to celiib
INFO - 2021-01-03 23:16:12,972 - settings - Setting database.password to newceliipass
INFO - 2021-01-03 23:16:12,976 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-03 23:16:12,976 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-03 23:16:12,987 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-03 23:16:13,221 - settings - Setting enable_python_native_blobs to True


In [3]:
test_mode = True

In [4]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-03 23:16:13,339 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-03 23:16:13,340 - settings - Setting database.user to celiib
INFO - 2021-01-03 23:16:13,341 - settings - Setting database.password to newceliipass
INFO - 2021-01-03 23:16:13,342 - settings - Setting enable_python_native_blobs to True


Sleeping 94 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-01-03 23:16:13,570 - settings - Setting enable_python_native_blobs to True


# Getting the list of neurons to decompose for the mutli soma testing

In [5]:
# import pandas as pd
# soma_soma_table = pd.read_csv("Minnie65 core proofreading - Soma-Soma.csv")
# no_header = soma_soma_table.iloc[1:]
# multi_soma_ids_str = no_header["Dendrites"].to_numpy()
# multi_soma_ids = multi_soma_ids_str[~np.isnan(multi_soma_ids_str.astype("float"))].astype("int")

# @schema
# class MultiSomaProofread(dj.Manual):
#     definition="""
#     segment_id : bigint unsigned  #segment id for those to be decimated
#     """
    
# dict_of_seg = [dict(segment_id=k) for k in multi_soma_ids]
# minnie.MultiSomaProofread.insert(dict_of_seg,skip_duplicates=True)
# MultiSomaProofread()

# Defining the Table

In [6]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [7]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 120


In [8]:
#so that it will have the adapter defined
from datajoint_utils import *

In [9]:
decimation_version = 0
decimation_ratio = 0.25
key_source = (minnie.Decimation().proj(decimation_version='version')  & 
              dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)  
              & minnie.MultiSomaProofread() & (dj.U("segment_id") & (minnie.BaylorSegmentCentroid() & "multiplicity=2").proj()))
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
864691134988402042,0,0.25
864691134988472442,0,0.25
864691135012400886,0,0.25
864691135081644023,0,0.25
864691135081658359,0,0.25
864691135103989069,0,0.25
864691135181738242,0,0.25
864691135181779714,0,0.25
864691135207768697,0,0.25
864691135210033344,0,0.25


In [10]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25

@schema
class Decomposition(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ---
    decomposition: <decomposition>
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branches
    
    run_time=NULL : double                   # the amount of time to run (seconds)

    
    """

#     key_source =  ((minnie.Decimation).proj(decimation_version='version') & 
#                             "decimation_version=" + str(decimation_version) &
#                        f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0" & "segment_id=864691136309663834").proj())
#     key_source = (minnie.Decimation() & "n_faces>500000").proj(decimation_version='version') & (minnie.BaylorSegmentCentroid() & "multiplicity=1").proj()

    
    key_source = (minnie.Decimation().proj(decimation_version='version')  & 
                  dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)  
                  & minnie.MultiSomaProofread() & (dj.U("segment_id") & (minnie.BaylorSegmentCentroid() & "multiplicity=2").proj()))

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decimated mesh
        3) Get the somas info
        4) Run the preprocessing
        5) Calculate all starter stats
        6) Save the file in a certain location
        7) Pass stats and file location to insert
        """
        whole_pass_time = time.time()
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        description = str(key['decimation_version']) + "_25"
        print(f"\n\n----- Working on {segment_id}-------")
        global_start = time.time()
        
        #2) Get the decimated mesh
        current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)

        #3) Get the somas info *************************** Need to change this when actually run *******************
        somas = du.get_soma_mesh_list(segment_id) 
        print(f"somas = {somas}")
        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"]

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Don't have to do any of the processing anymore because will do in the neuron object
        stats_dict = recovered_neuron.neuron_stats()



        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")



        #7) Pass stats and file location to insert
        new_key = dict(key,
                       decomposition=ret_file_path_str,
                       n_vertices=len(current_neuron_mesh.vertices),
                       n_faces=len(current_neuron_mesh.faces),
                       run_time=np.round(time.time() - whole_pass_time,4)
                      )
        new_key.update(stats_dict)

        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
    

# Running the Populate

In [24]:
minnie.Decomposition()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),decomposition,n_vertices number of vertices,n_faces number of faces,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)


In [25]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'")
curr_table
#curr_table.delete()
#(curr_table & "error_message = 'ValueError: need at least one array to concatenate'").delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition,0346046686f7bf941663a32a3d275407,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.154,at-node27,29,21108,2021-01-03 22:39:12
__decomposition,07568ee6e54a5734b7a255efc6817437,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.175,at-node48,29,21130,2021-01-03 22:41:06
__decomposition,15561dcd3819dda18d335bd3505428ee,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.81,at-compute003,29,21092,2021-01-03 22:44:13
__decomposition,1dabd9be0656c837a3baac4e2113b48b,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.164,at-node37,29,21102,2021-01-03 22:37:10
__decomposition,22efd0541a14a713e292d1e6da2e2962,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.130,at-node3,29,21119,2021-01-03 22:34:00
__decomposition,26f6cebdc9e967351f8d399546dcc588,error,=BLOB=,Exception: Something went wrong when trying to find end nodes,=BLOB=,celiib@10.28.0.157,at-node30,29,21099,2021-01-03 22:51:52
__decomposition,2be267f7a1a0ae3a64209322665950ab,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.81,at-compute003,29,21098,2021-01-03 22:43:43
__decomposition,3a002ead65c7367cc6259695157121d4,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.152,at-node25,29,21111,2021-01-03 22:34:50
__decomposition,3b06de84ce60b58b475eee322e774180,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.145,at-node18,29,21115,2021-01-03 22:42:22
__decomposition,3f25521280102fcb7a5da8085dc0dc86,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.81,at-compute003,30,21097,2021-01-03 22:41:35


In [27]:
(curr_table & "key_hash='26f6cebdc9e967351f8d399546dcc588'").fetch("key")

array([{'segment_id': 864691135269687205, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}],
      dtype=object)

In [22]:
curr_table.delete()

In [ ]:
possible_keys = [
"dd46cdbc9b22b05e1a6b60e456cf93f6",
"26f6cebdc9e967351f8d399546dcc588",
"0346046686f7bf941663a32a3d275407"
]

In [ ]:
cur_error = (curr_table & dict(key_hash = possible_keys[0]))
cur_error

In [ ]:
cur_error

In [ ]:
# import pandas as pd
# key_hash,error_message = curr_table.fetch("key_hash","error_message")

# df = pd.DataFrame.from_dict([dict(key_hash = k,error_message = m) for k,m in zip(key_hash,error_message)])
# df
# #df.columns = ["error","key_hash"]
# key_hashes_to_delete = df[df["error_message"].str.contains("OSError")]["key_hash"].to_numpy()

# (curr_table & [dict(key_hash=k) for k in key_hashes_to_delete]).delete()

In [ ]:
import time
import random
import compartment_utils as cu
cu = reload(cu)
import preprocessing_vp2 as pre
pre = reload(pre)
import skeleton_utils as sk
sk = reload(sk)
nru = reload(nru)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    Decomposition.populate(reserve_jobs=True, suppress_errors=True)
else:
    Decomposition.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for DecompositionMultiSoma populate = {time.time() - start_time}")

In [ ]:
debug